In [ ]:
import numpy as np
import pandas as pd

In [ ]:
housing = pd.read_csv('datasets/housing.csv')
housing.Co

In [ ]:
housing.info()

In [ ]:
corr = housing.corr()

In [ ]:
type(corr)

In [ ]:
corr.columns

In [ ]:
corr.index

In [ ]:
corr

In [ ]:
corr['median_house_value'].sort_values(ascending=False)

In [ ]:
corr.loc['median_house_value', :].sort_values(ascending=False)

In [ ]:
np.ceil(housing['median_income'][:20] / 1.5)

In [ ]:
housing['income_cat'] = np.ceil(housing['median_income'] / 1.5)

In [ ]:
housing['income_cat'].where(housing['income_cat'] < 5.0, 5.0, inplace=True)

In [ ]:
housing['income_cat'][:20]

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [11]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [12]:
gen = split.split(housing, housing['income_cat'])
indices = list(gen)

NameError: name 'housing' is not defined

In [13]:
strat_train_set = housing.loc[indices[0][0]]
strat_test_set = housing.loc[indices[0][1]]

NameError: name 'housing' is not defined

In [14]:
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

NameError: name 'housing' is not defined

In [15]:
strat_train_set['income_cat'].value_counts() / len(strat_train_set)

NameError: name 'strat_train_set' is not defined

In [16]:
strat_test_set['income_cat'].value_counts() / len(strat_test_set)

NameError: name 'strat_test_set' is not defined

In [17]:
housing['income_cat'].value_counts() / len(housing)

NameError: name 'housing' is not defined

In [18]:
strat_train_set.drop('income_cat', axis=1, inplace=True)

NameError: name 'strat_train_set' is not defined

In [19]:
strat_test_set.drop('income_cat', axis=1, inplace=True)

NameError: name 'strat_test_set' is not defined

In [20]:
housing = strat_train_set.drop('median_house_value', axis=1)

NameError: name 'strat_train_set' is not defined

In [21]:
housing_labels = strat_train_set['median_house_value'].copy()

NameError: name 'strat_train_set' is not defined

In [22]:
housing.info()

NameError: name 'housing' is not defined

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

NameError: name 'housing' is not defined

In [24]:
housing_extra_attribs.shape

NameError: name 'housing_extra_attribs' is not defined

In [25]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [26]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [27]:
list(housing)

NameError: name 'housing' is not defined

In [28]:
from sklearn.preprocessing import Imputer, LabelBinarizer, StandardScaler

In [29]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(list(housing)[:-1])),
    ('imputer', Imputer(strategy='median')),
    ('adder', CombinedAttributesAdder()),
    ('scalar', StandardScaler())
])

NameError: name 'housing' is not defined

In [30]:
from future_encoders import OneHotEncoder
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(['ocean_proximity'])),
#    ('binarizer', LabelBinarizer())
    ('binarizer', OneHotEncoder(sparse=False))
])

ModuleNotFoundError: No module named 'future_encoders'

In [31]:
full_pipeline = FeatureUnion([
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline)
])

NameError: name 'num_pipeline' is not defined

In [32]:
housing_prepared = full_pipeline.fit_transform(housing)

NameError: name 'full_pipeline' is not defined

In [33]:
housing_prepared.shape

NameError: name 'housing_prepared' is not defined

In [34]:
from sklearn.model_selection import cross_val_score

In [35]:
from sklearn.ensemble import RandomForestRegressor

C:\Users\sumit\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [36]:
model = RandomForestRegressor()

In [37]:
scores = cross_val_score(model, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

NameError: name 'housing_prepared' is not defined

In [ ]:
scores

In [ ]:
rmse = np.sqrt(-scores)

In [ ]:
rmse

In [ ]:
param_grid = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring="neg_mean_squared_error")

In [ ]:
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
scores = cross_val_score(grid_search.best_estimator_, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

In [38]:
rmse = np.sqrt(-scores)

NameError: name 'scores' is not defined

In [39]:
rmse

NameError: name 'rmse' is not defined

In [40]:
from sklearn.model_selection import RandomizedSearchCV

In [41]:
params = {'n_estimators': [i for i in range(1, 100)], 'max_features': [i for i in range(1, 10)]}

In [42]:
rand_search = RandomizedSearchCV(model, params, cv=5, scoring="neg_mean_squared_error")

In [43]:
rand_search.fit(housing_prepared, housing_labels)

NameError: name 'housing_prepared' is not defined

In [44]:
rand_search.best_params_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [45]:
rand_search.best_estimator_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [46]:
scores = cross_val_score(rand_search.best_estimator_, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [47]:
rmse = np.sqrt(-scores)

NameError: name 'scores' is not defined

In [48]:
rmse

NameError: name 'rmse' is not defined

In [49]:
model = rand_search.best_estimator_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [50]:
model.fit(housing_prepared, housing_labels)

NameError: name 'housing_prepared' is not defined

In [51]:
test_set = strat_test_set.drop('median_house_value', axis=1)

NameError: name 'strat_test_set' is not defined

In [52]:
test_labels = strat_test_set['median_house_value'].copy()

NameError: name 'strat_test_set' is not defined

In [53]:
test_set_prepared = full_pipeline.fit_transform(test_set)

NameError: name 'full_pipeline' is not defined

In [54]:
predictions = model.predict(test_set_prepared)

NameError: name 'test_set_prepared' is not defined

In [55]:
from sklearn.metrics import mean_squared_error

In [56]:
mse = mean_squared_error(test_labels, predictions)

NameError: name 'test_labels' is not defined

In [57]:
rmse = np.sqrt(mse)

NameError: name 'mse' is not defined

In [58]:
rmse

NameError: name 'rmse' is not defined